Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## TODO

* Wrap all Exchange Order responses in an Order object
* Market orders do not return price. So fetch trades. https://github.com/ccxt/ccxt/issues/798
* Verify starting_cash <= actual balance on Live Exchange
* Handle/prevent 'insufficient funds' from Binance/Poloniex (Above will help, but still need to catch if the exchange balance has changed for some reason)
* Handle/prevent 'minimum order not met' from Binanace/Poloniex (method to refresh and lookup the minimum amount)
* Method to audit/confirm our local exchange balance reflects actual changes to CCXT exchange balance
* Limit Sell Orders not working I think..


## Define Strategy

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

class SimpleStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def log_all(self, orders, data, ctx, time_utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(time_utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        asset = Asset(c.ETH, c.BTC)
        price = data['close']
        quantity = .01

        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, asset, quantity, price)
            orders.append(order)
        elif (len(ctx.record.portfolio.positions) > 0 
              and ctx.record.balance.get(c.ETH)[BalanceType.FREE] > 0.0):
            order = order_manager.build_market_sell_order(
                ctx.exchange, asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []
        
        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)

        self.log_all(orders, data, ctx, data['time_utc'])
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [ ]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = CSVDataFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
strategy = SimpleStrategy()
experiment_name = 'default_backtest'

In [ ]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

## Simulate Orders

In [2]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.ETH, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_simulate'

In [ ]:
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

Downloading: ETH/BTC
Downloaded rows: 1
Downloading: ETH/BTC
Downloaded rows: 1
No data after prior poll: 2018-01-18 07:16:16.651764
Downloading: ETH/BTC
Downloaded rows: 1
No data after prior poll: 2018-01-18 07:16:16.651764
Downloading: ETH/BTC


---------------------------------------
Epoch 1 - Timestep: 2018-01-18T07:18:00
---------------------------------------
OHLCV
    O: 0.0898 | C: 0.0899 | V: 43.7 | T: 2018-01-18T07:18:00
ORDERS
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 2
Downloading: ETH/BTC
Downloaded rows: 1
No data after prior poll: 2018-01-18 07:18:00
Downloading: ETH/BTC
Downloaded rows: 2


---------------------------------------
Epoch 2 - Timestep: 2018-01-18T07:19:00
---------------------------------------
OHLCV
    O: 0.0899 | C: 0.0901 | V: 125.0 | T: 2018-01-18T07:19:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0901 | Quantity: 0.0100
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


Placing Order {
    "id": "e07ff2bd158242d7b77422677fa0a869",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.09013,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-18T07:19:20.763667",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "e07ff2bd158242d7b77422677fa0a869",
    "exchange_id": "paper",
    "exchange_order_id": "41dabd073229497fb2913271f2a4dcfb",
    "price": 0.09013,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-18T07:19:20.763667",
    "opened_time": null,
    "filled_time": "2018-01-18T07:19:20.812516",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Downloading: ETH/BTC
Downloaded rows: 1
N

...

---------------------------------------
Epoch 21 - Timestep: 2018-01-18T07:38:00
---------------------------------------
OHLCV
    O: 0.0895 | C: 0.0895 | V: 3.9 | T: 2018-01-18T07:38:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9964 Total Val: 1.0000 PnL: -0.0000 Returns: -0.0000
BALANCE
    BTC - {'free': 0.99281554000000027, 'used': 0.0, 'total': 0.99281554000000027}
    ETH - {'free': 0.08, 'used': 0.0, 'total': 0.08}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.04, 'cost_price': 0.08960466666666668, 'latest_price': 0.089500999999999997}
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 2
Placing Order {
    "id": "d8270465d6d34bb6b6d631f85095c10a",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "CREATED",
    "created_time": "2018-01-18T07:38:04.202025",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "d8270465d6d34bb6b6d631f85095c10a",
    "exchange_id": "paper",
    "exchange_order_id": "16c77eb111944f2ca0a1f121316ef40d",
    "price": 0.0895,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "MARKET_SELL",
    "status": "FILLED",
    "created_time": "2018-01-18T07:38:04.202025",
    "opened_time": null,
    "filled_time": "2018-01-18T07:38:04.401067",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Downloading: ETH/BTC
Do

---------------------------------------
Epoch 22 - Timestep: 2018-01-18T07:39:00
---------------------------------------
OHLCV
    O: 0.0896 | C: 0.0895 | V: 9.9 | T: 2018-01-18T07:39:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9973 Total Val: 1.0000 PnL: -0.0000 Returns: -0.0000
BALANCE
    BTC - {'free': 0.99460554000000023, 'used': 0.0, 'total': 0.99460554000000023}
    ETH - {'free': 0.060000000000000005, 'used': 0.0, 'total': 0.060000000000000005}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.03, 'cost_price': 0.08960466666666668, 'latest_price': 0.089499999999999996}
METRICS
    SMA: 5.00
    RSI: 10.00


Placing Order {
    "id": "175cd58083d64179916b16b17e69337a",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "CREATED",
    "created_time": "2018-01-18T07:39:04.986786",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "175cd58083d64179916b16b17e69337a",
    "exchange_id": "paper",
    "exchange_order_id": "bf3357d087854357a8d5a250c17bc4e0",
    "price": 0.089501,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "MARKET_SELL",
    "status": "FILLED",
    "created_time": "2018-01-18T07:39:04.986786",
    "opened_time": null,
    "filled_time": "2018-01-18T07:39:05.185272",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Downloading: ETH/BTC
Downloaded rows: 1


RequestTimeout: binance GET https://api.binance.com/api/v1/klines?symbol=ETHBTC&interval=1m&limit=500 request timeout

## Live Trade

In [ ]:
def live(name, exchange, balance, portfolio, feed, strategy):
    print("WARNING LIVE TRADING!!!!")
    '''
    exp_name = name of your current experiment (multiple runs per strategy)
    '''
    # Where we will save the record
    root = os.path.join(cfg.DATA_DIR, name)

    # This can be retrieved from the user's global config
    store = DATA_STORES[cfg.DATA_STORE](root=root)

    config = {
        'experiment': name, 
        'strategy': strategy.name,
    }
    record = Record(
        config=config,
        portfolio=portfolio,
        balance=balance,
        store=store
    )
    ctx = Context(
        exchange=exchange,
        feed=feed,
        record=record
    )
    feed.initialize()
    
    while True:
        row = feed.next()
        
        if row is not None:
            orders = strategy.process(row, ctx)

            # TODO: Cancelling orders
            # should we auto-cancel any outstanding orders
            # or should we leave this decision up to the Strategy?
            order_manager.cancel_orders(exchange, orders['cancel_ids'])

            # Returns both FILLED and PENDING orders
            # TODO: Order manager handles mapping from Exchange JSON
            # Particularly order types like CLOSED --> FILLED,
            # And OPEN vs PENDING <-- check the 'quantity' vs 'filled' amounts
            orders = order_manager.place_orders(exchange, orders['orders'])
            filled_orders = order_manager.get_filled_orders(orders)

            # Portfolio needs to know about new filled orders
            portfolio.update(filled_orders)

            # Getting the latest prices for each of our positions
            latest_prices = get_latest_prices(portfolio.positions, row)
            portfolio.update_position_prices(latest_prices)


            # Record needs to know about all new orders
            for order in orders:
                record.orders[order.id] = order

            # Update Virtual Balance
            # exchange balance may be impacted by external trading
            for order in filled_orders:
                balance.update_by_order(order)

            record.save()
        
        time.sleep(30)
            
    return record

In [ ]:
cash = .04
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_exchange(c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.XRP, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_live'

In [ ]:
live(experiment_name, exchange, balance, portfolio, feed, strategy)